# importing

In [3]:
import os
os.environ["KERAS_BACKEND"] = "plaidml.keras.backend"

os.environ["RUNFILES_DIR"] = "/Users/bunnykitty/opt/anaconda3/envs/tf/share/plaidml"
# plaidml might exist in different location. Look for "/usr/local/share/plaidml" and replace in above path

os.environ["PLAIDML_NATIVE_PATH"] = "/Users/bunnykitty/opt/anaconda3/envs/tf/lib/libplaidml.dylib"
# libplaidml.dylib might exist in different location. Look for "/usr/local/lib/libplaidml.dylib" and replace in above path

In [4]:
# LSTM with dropout for sequence classification in the IMDB dataset
import numpy as np
from numpy import array
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.layers import Dropout
from keras.layers import TimeDistributed
from keras.layers import Activation


# hyperparameter  setting

# Data generating

In [5]:
# 产生一长串卷积数据，然后切割成batch (不考虑state memory，即由keras自动划分epoch)
# fix random seed for reproducibility
np.random.seed(15)
# load the dataset but only keep the top n words, zero the rest
#epochs=3
#sample_epoch=2000

#samples=batch_size*epochs
#samples=5000                                             # train_set size=samples*seq_len/train_test_ratio  train_test_ratio=3 
seq_len=10                                            # time-step size
lstm_size=100
dropout=0


epochs_step=10
data_reps=5

batch_size=500
shuffle_option=False



train_sample=5000
train_test_ratio=1

datasize=train_sample*train_test_ratio

samples_iter=1000

snr_db=20


#datasize=seq_len*samples
#datasize=10

modulation=2                #modulation^2 为 QAM星座点数

h0=np.array([0.5 ], dtype=complex)                            #awgn
h1=np.array([0.97, 0.23, 0.45, 0.11  ], dtype=complex)
h2=np.array([0.3482, 0.8704, 0.3482  ], dtype=complex)  # Linear non-minimum phase channel.
h4=np.array([1.22 + 1j*0.646,    0.063 - 1j*0.001,     -0.024 - 1j*0.014,    0.036 + 1j*0.031 ],dtype=complex)  #from Adaptive Decision Feedback Equalization for Digital
h3=np.array([1, -2, 1   ], dtype=complex)                # partial response channel with double zero on the unit circle

channel_h=h2

In [6]:


source_0=np.random.randint(0,modulation,datasize*2)            #16qam     0 1 2 3
source=np.array(source_0)
#print(source)
#source=source*2-3                                       # -3 -1 1 3
source=source*2-(modulation-1)                          # -1 1
source_0=np.reshape(source_0,[-1,2])                  # split into real and imag part
source=np.reshape(source,[-1,2])                      # split into real and imag part
#print(source)
print("source code size is:" , source.shape)
#print(source[0:5,:])

#out_demod=source_0[:,0]*4+source_0[:,1]            # complex modulation to 0-15 constellation point
out_demod=source_0[:,0]*modulation+source_0[:,1]            # complex modulation to 0-15 constellation point
#print(out_demod)
print("out_demod len=",len(out_demod))
out_demod_cat=np.zeros((len(out_demod),16))

for i in range(len(out_demod)):
    out_demod_cat[i,out_demod[i]]=1

print("out_demod_cat size",out_demod_cat.shape)




#print(out_demod)

#out_demod=np.reshape(out_demod,[samples,-1])
#print("out_demod shape=",out_demod.shape)
#print(out_demod)
#out_demod=sequence.pad_sequences(out_demod, seq_len+3, padding='post', truncating='pre', value=16.0)
#print(out_demod)
#print("out_demod size is:" , out_demod.shape)




source code size is: (5000, 2)
out_demod len= 5000
out_demod_cat size (5000, 16)


In [7]:
source_z=source[:,0]+1j*source[:,1]


snr=10**(snr_db/10)


receive_z_1=np.convolve(source_z, channel_h,'same')        # 一整串卷积 然后切割
#receive_z_1=receive_z_1+0.2*(receive_z_1**2)  # 非线性信道

#print(receive_z_1[0:20])

power=np.square(np.absolute(receive_z_1))                  # 对输入采样进行归一化
#print(power[0:20])


signal_power=np.average(power)                            #添加噪声
print("signal power=", signal_power)

noise_oneside_power=signal_power/2/snr
#print(noise_oneside_power)
noise=np.random.normal(0, noise_oneside_power**0.5, receive_z_1.shape)+1j*np.random.normal(0, noise_oneside_power**0.5, receive_z_1.shape)
#noise=np.random.normal(0, 1, receive_z_1.shape)+1j*np.random.normal(0, 1, receive_z_1.shape)
#print(noise[0:20])
print("complex noise average =",np.average(noise))
print("complex noise power =", np.average(np.square(np.absolute(noise))))
print("snr", signal_power/(np.average(np.square(np.absolute(noise)))) )

receive_z_1_with_noise=receive_z_1+noise
print("  recieved no noise", receive_z_1[0:4])
print("recieved with noise", receive_z_1_with_noise[0:4])

print("              noise", receive_z_1_with_noise[0:4]-receive_z_1[0:4])

power_with_noise=np.square(np.absolute(receive_z_1_with_noise))                  # 对输入采样进行归一化 scaling
#print(power[0:20])

scale=(np.amax(power_with_noise))**0.5

print("scaling factor=", scale)


receive_z_1_with_noise=receive_z_1_with_noise/scale

#receive_z_1=np.reshape(receive_z_1,[samples,-1])

# 这个方法暂时不用了，就使用上面的整串卷积
# source_z2=np.reshape(source_z,[samples,-1])       # 先切割成片段 然后卷积  
# receive_z_2=[]
# #print(source_z2.shape[0])
# for l in range(source_z2.shape[0]):
#   temp=np.convolve(source_z2[l,:],channel_h,'same')
#   receive_z_2=np.append(receive_z_2, temp)


# print(receive_z_1)
# print(receive_z_2.shape)
# print(receive_z_1-receive_z_2)



receive_z=receive_z_1_with_noise
#print("receive_z = ", receive_z)
print("receive_z shape= ", receive_z.shape)

input_temp=np.array((receive_z.real, receive_z.imag)).T         # 分成实部虚部
#print("input_temp = ",input_temp)
print("input_temp shape= ",input_temp.shape)
#print(np.append([np.zeros((seq_len,2)),np.zeros((seq_len,2))],input_temp[20:(seq_len+20),:].reshape(1,seq_len,2),axis=0))


samples=input_temp.shape[0]-seq_len           # N samples 只能产生 N-seq_len 的训练数据，除非在samples后面填0
print("samples length=", samples)
input_z=np.zeros((seq_len,2))
#input_z=np.empty([seq_len,2])
input_z=input_z.reshape(1,seq_len,2)

for i in range(samples):
  #input_z[i,:,:]=input_temp[i:(seq_len+i),:].reshape(1,seq_len,2)
  input_z=np.append(input_z, input_temp[i:(seq_len+i),:].reshape(1,seq_len,2), axis=0)
  #print(input_z)
  
input_z=input_z[1:,:,:]
#input_z=np.reshape(input_z,[samples,seq_len,2])
#print("input_z = ",input_z)
print("input_z shape= ",input_z.shape)



out_shift=seq_len//2+1                           #单个输出，与偏移后输入符号做判决，偏移量，一般选中间位置
out_demod=out_demod[(0+out_shift):(samples+out_shift),]                         # [8]      1位输出  sparse_categorical_crossentropy
print("out_demod size =", out_demod.shape)
#out_demod_cat=out_demod_cat[0:samples,]                # [0,1,0,0,0.....,0]  16位输出     'categorical_crossentropy

#print("out_demod_cat size",out_demod_cat.shape)


signal power= 2.0288723922239997
complex noise average = (0.0012062503103275957+0.0004250548603093989j)
complex noise power = 0.0202880633919003
snr 100.00325575845727
  recieved no noise [-1.2186+1.2186j -0.8704+0.8704j  0.174 -0.174j  -0.174 +0.8704j]
recieved with noise [-1.33087599+1.19600152j -0.96344476+0.70384026j  0.18515314-0.09335366j
 -0.11571473+0.82885984j]
              noise [-0.11227599-0.02259848j -0.09304476-0.16655974j  0.01115314+0.08064634j
  0.05828527-0.04154016j]
scaling factor= 2.597455662890496
receive_z shape=  (5000,)
input_temp shape=  (5000, 2)
samples length= 4990
input_z shape=  (4990, 10, 2)
out_demod size = (4990,)


In [8]:
input_z = input_z
y_output=out_demod

#train_test_ratio=10
X_train = input_z[0:samples//train_test_ratio,:,:]
#X_test = input_z[samples//train_test_ratio:,:,:]        #第一组数据全部作为test 数据
X_test0 = input_z

y_train = y_output[0:samples//train_test_ratio]
#y_test  = y_output[samples//train_test_ratio:]
y_test0  = y_output

print(X_train.shape)
print(X_test0.shape)
print(y_train.shape)
print(y_test0.shape)

(4990, 10, 2)
(4990, 10, 2)
(4990,)
(4990,)


# model setting

In [9]:

# create the model
model = Sequential()
model.add(LSTM(lstm_size, input_shape=(X_train.shape[1], X_train.shape[2]), dropout=dropout))
#model.add(Dropout(0.2))
#model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
#model.add(Dense(16, activation='softmax'))

#model.add(Dense(10, activation='sigmoid'))
model.add(Dense(modulation**2, activation='softmax'))
#model.add((Dense(16)))
#model.add(Activation('softmax'))

#model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
#model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['categorical_accuracy'])
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])  #sparse_categorical_accuracy

print(model.summary())



INFO:plaidml:Opening device "metal_amd_radeon_pro_5500m.0"


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 100)               41200     
_________________________________________________________________
dense_1 (Dense)              (None, 4)                 404       
Total params: 41,604
Trainable params: 41,604
Non-trainable params: 0
_________________________________________________________________
None


# trainingg and validating

In [10]:

train_sample=samples_iter
train_test_ratio=2
datasize=train_sample*train_test_ratio

data_set=0
for i in range(data_reps):
    print(" SNR in db:", snr_db)
    print("modulation:", modulation**2)
    print("channel im:", channel_h)
    print("LSTM size :", lstm_size)
    print("batchsize :", batch_size)
    print("   dropout:", dropout)
    print("===================================================================================")

    source_0=np.random.randint(0,modulation,datasize*2)            #16qam     0 1 2 3
    source=np.array(source_0)
    source=source*2-(modulation-1)                          # -1 1
    source_0=np.reshape(source_0,[-1,2])                  # split into real and imag part
    source=np.reshape(source,[-1,2])                      # split into real and imag part
    out_demod=source_0[:,0]*modulation+source_0[:,1]            # complex modulation to 0-15 constellation point
    out_demod_cat=np.zeros((len(out_demod),16))

    for i in range(len(out_demod)):
        out_demod_cat[i,out_demod[i]]=1

    source_z=source[:,0]+1j*source[:,1]

    receive_z_1=np.convolve(source_z, channel_h,'same')        # 一整串卷积 然后切割
  #receive_z_1=receive_z_1+0.2*(receive_z_1**2)  # 非线性信道


    power=np.square(np.absolute(receive_z_1))                  # 对输入采样进行归一化



    signal_power=np.average(power)                            #添加噪声


    noise_oneside_power=signal_power/2/snr

    noise=np.random.normal(0, noise_oneside_power**0.5, receive_z_1.shape)+1j*np.random.normal(0, noise_oneside_power**0.5, receive_z_1.shape)

#   print("complex noise average =",np.average(noise))
#   print("complex noise power =", np.average(np.square(np.absolute(noise))))
#   print("snr", signal_power/(np.average(np.square(np.absolute(noise)))) )

    receive_z_1_with_noise=receive_z_1+noise
#   print(receive_z_1_with_noise[0:4])
#   print(receive_z_1_with_noise[0:4]-receive_z_1[0:4])

    power_with_noise=np.square(np.absolute(receive_z_1_with_noise))                  # 对输入采样进行归一化 scaling


  #scale=(np.amax(power_with_noise))**0.5
    receive_z_1_with_noise=receive_z_1_with_noise/scale
    receive_z=receive_z_1_with_noise

    input_temp=np.array((receive_z.real, receive_z.imag)).T         # 分成实部虚部



    samples=input_temp.shape[0]-seq_len           # N samples 只能产生 N-seq_len 的训练数据，除非在samples后面填0

    input_z=np.zeros((seq_len,2))

    input_z=input_z.reshape(1,seq_len,2)

    for i in range(samples):
        input_z=np.append(input_z, input_temp[i:(seq_len+i),:].reshape(1,seq_len,2), axis=0)

    input_z=input_z[1:,:,:]

    out_shift=seq_len//2+1                           #单个输出，与偏移后输入符号做判决，偏移量，一般选中间位置
    out_demod=out_demod[(0+out_shift):(samples+out_shift),]                         # [8]      1位输出  sparse_categorical_crossentropy

    input_z = input_z
    y_output=out_demod

    X_train = input_z[0:samples//train_test_ratio,:,:]
    X_test = input_z[samples//train_test_ratio:,:,:]        


    y_train = y_output[0:samples//train_test_ratio]
    y_test  = y_output[samples//train_test_ratio:]

    epochs_step = 100
    model.fit(X_train, y_train, validation_data=(X_test,y_test), epochs=epochs_step, batch_size=batch_size, shuffle=shuffle_option, verbose=1)


  # Final evaluation of the model
    scores = model.evaluate(X_test0, y_test0, verbose=1)
  #scores = model.evaluate(X_test, y_test, verbose=1)
    print("========================================================= \n  Accuracy: %.2f%%" % (scores[1]*100))
    data_set+=1
    print("  data set: %d / %d " %(data_set,data_reps))
#   print(" SNR in db:", snr_db)
#   print("===================================================================================")

 SNR in db: 20
modulation: 4
channel im: [0.3482+0.j 0.8704+0.j 0.3482+0.j]
LSTM size : 100
batchsize : 500
   dropout: 0
Train on 995 samples, validate on 995 samples
Epoch 1/100
995/995 [==============================] - 8s 8ms/step - loss: 1.3809 - acc: 0.2995 - val_loss: 1.3752 - val_acc: 0.3698
Epoch 2/100
995/995 [==============================] - 0s 44us/step - loss: 1.3728 - acc: 0.3598 - val_loss: 1.3682 - val_acc: 0.4271
Epoch 3/100
995/995 [==============================] - 0s 47us/step - loss: 1.3650 - acc: 0.4020 - val_loss: 1.3613 - val_acc: 0.4412
Epoch 4/100
995/995 [==============================] - 0s 50us/step - loss: 1.3571 - acc: 0.4070 - val_loss: 1.3542 - val_acc: 0.4322
Epoch 5/100
995/995 [==============================] - 0s 46us/step - loss: 1.3490 - acc: 0.4151 - val_loss: 1.3469 - val_acc: 0.4302
Epoch 6/100
995/995 [==============================] - 0s 49us/step - loss: 1.3408 - acc: 0.4111 - val_loss: 1.3395 - val_acc: 0.4322
Epoch 7/100
995/995 [========

Epoch 60/100
995/995 [==============================] - 0s 48us/step - loss: 0.8300 - acc: 0.6281 - val_loss: 0.8839 - val_acc: 0.6101
Epoch 61/100
995/995 [==============================] - 0s 49us/step - loss: 0.8218 - acc: 0.6191 - val_loss: 0.8704 - val_acc: 0.6161
Epoch 62/100
995/995 [==============================] - 0s 48us/step - loss: 0.8169 - acc: 0.6261 - val_loss: 0.8549 - val_acc: 0.6211
Epoch 63/100
995/995 [==============================] - 0s 50us/step - loss: 0.8046 - acc: 0.6362 - val_loss: 0.8519 - val_acc: 0.6171
Epoch 64/100
995/995 [==============================] - 0s 46us/step - loss: 0.7952 - acc: 0.6422 - val_loss: 0.8449 - val_acc: 0.6352
Epoch 65/100
995/995 [==============================] - 0s 46us/step - loss: 0.7913 - acc: 0.6322 - val_loss: 0.8309 - val_acc: 0.6251
Epoch 66/100
995/995 [==============================] - 0s 48us/step - loss: 0.7843 - acc: 0.6573 - val_loss: 0.8229 - val_acc: 0.6312
Epoch 67/100
995/995 [==============================] -

Epoch 18/100
995/995 [==============================] - 0s 45us/step - loss: 0.3981 - acc: 0.8291 - val_loss: 0.3987 - val_acc: 0.8372
Epoch 19/100
995/995 [==============================] - 0s 43us/step - loss: 0.3892 - acc: 0.8412 - val_loss: 0.3895 - val_acc: 0.8452
Epoch 20/100
995/995 [==============================] - 0s 46us/step - loss: 0.3797 - acc: 0.8432 - val_loss: 0.3798 - val_acc: 0.8503
Epoch 21/100
995/995 [==============================] - 0s 44us/step - loss: 0.3704 - acc: 0.8492 - val_loss: 0.3722 - val_acc: 0.8543
Epoch 22/100
995/995 [==============================] - 0s 43us/step - loss: 0.3626 - acc: 0.8553 - val_loss: 0.3647 - val_acc: 0.8583
Epoch 23/100
995/995 [==============================] - 0s 44us/step - loss: 0.3541 - acc: 0.8533 - val_loss: 0.3535 - val_acc: 0.8633
Epoch 24/100
995/995 [==============================] - 0s 44us/step - loss: 0.3430 - acc: 0.8623 - val_loss: 0.3424 - val_acc: 0.8673
Epoch 25/100
995/995 [==============================] -

Epoch 79/100
995/995 [==============================] - 0s 50us/step - loss: 0.0594 - acc: 0.9910 - val_loss: 0.0635 - val_acc: 0.9819
Epoch 80/100
995/995 [==============================] - 0s 46us/step - loss: 0.0575 - acc: 0.9920 - val_loss: 0.0621 - val_acc: 0.9819
Epoch 81/100
995/995 [==============================] - 0s 54us/step - loss: 0.0561 - acc: 0.9920 - val_loss: 0.0605 - val_acc: 0.9829
Epoch 82/100
995/995 [==============================] - 0s 48us/step - loss: 0.0543 - acc: 0.9920 - val_loss: 0.0585 - val_acc: 0.9849
Epoch 83/100
995/995 [==============================] - 0s 45us/step - loss: 0.0524 - acc: 0.9930 - val_loss: 0.0570 - val_acc: 0.9849
Epoch 84/100
995/995 [==============================] - 0s 44us/step - loss: 0.0509 - acc: 0.9940 - val_loss: 0.0557 - val_acc: 0.9849
Epoch 85/100
995/995 [==============================] - 0s 43us/step - loss: 0.0495 - acc: 0.9940 - val_loss: 0.0542 - val_acc: 0.9849
Epoch 86/100
995/995 [==============================] -

Epoch 37/100
995/995 [==============================] - 0s 45us/step - loss: 0.0230 - acc: 0.9960 - val_loss: 0.0555 - val_acc: 0.9789
Epoch 38/100
995/995 [==============================] - 0s 50us/step - loss: 0.0308 - acc: 0.9930 - val_loss: 0.0655 - val_acc: 0.9749
Epoch 39/100
995/995 [==============================] - 0s 52us/step - loss: 0.0275 - acc: 0.9930 - val_loss: 0.0731 - val_acc: 0.9739
Epoch 40/100
995/995 [==============================] - 0s 50us/step - loss: 0.0426 - acc: 0.9869 - val_loss: 0.0757 - val_acc: 0.9729
Epoch 41/100
995/995 [==============================] - 0s 51us/step - loss: 0.0311 - acc: 0.9910 - val_loss: 0.1031 - val_acc: 0.9578
Epoch 42/100
995/995 [==============================] - 0s 47us/step - loss: 0.0585 - acc: 0.9759 - val_loss: 0.1215 - val_acc: 0.9548
Epoch 43/100
995/995 [==============================] - 0s 48us/step - loss: 0.0517 - acc: 0.9789 - val_loss: 0.2018 - val_acc: 0.9146
Epoch 44/100
995/995 [==============================] -

Epoch 98/100
995/995 [==============================] - 0s 45us/step - loss: 0.0065 - acc: 1.0000 - val_loss: 0.0203 - val_acc: 0.9970
Epoch 99/100
995/995 [==============================] - 0s 47us/step - loss: 0.0064 - acc: 1.0000 - val_loss: 0.0202 - val_acc: 0.9970
Epoch 100/100
4990/4990 [==============================] - 1s 158us/step
  Accuracy: 99.64%
  data set: 3 / 5 
 SNR in db: 20
modulation: 4
channel im: [0.3482+0.j 0.8704+0.j 0.3482+0.j]
LSTM size : 100
batchsize : 500
   dropout: 0
Train on 995 samples, validate on 995 samples
Epoch 1/100
995/995 [==============================] - 0s 44us/step - loss: 0.0234 - acc: 0.9930 - val_loss: 0.0159 - val_acc: 0.9950
Epoch 2/100
995/995 [==============================] - 0s 45us/step - loss: 0.0219 - acc: 0.9930 - val_loss: 0.0149 - val_acc: 0.9970
Epoch 3/100
995/995 [==============================] - 0s 44us/step - loss: 0.0201 - acc: 0.9960 - val_loss: 0.0146 - val_acc: 0.9970
Epoch 4/100
995/995 [============================

Epoch 56/100
995/995 [==============================] - 0s 46us/step - loss: 0.0065 - acc: 1.0000 - val_loss: 0.0078 - val_acc: 0.9990
Epoch 57/100
995/995 [==============================] - 0s 44us/step - loss: 0.0064 - acc: 1.0000 - val_loss: 0.0077 - val_acc: 1.0000
Epoch 58/100
995/995 [==============================] - 0s 44us/step - loss: 0.0064 - acc: 1.0000 - val_loss: 0.0077 - val_acc: 1.0000
Epoch 59/100
995/995 [==============================] - 0s 44us/step - loss: 0.0063 - acc: 1.0000 - val_loss: 0.0076 - val_acc: 1.0000
Epoch 60/100
995/995 [==============================] - 0s 44us/step - loss: 0.0062 - acc: 1.0000 - val_loss: 0.0076 - val_acc: 1.0000
Epoch 61/100
995/995 [==============================] - 0s 47us/step - loss: 0.0062 - acc: 1.0000 - val_loss: 0.0075 - val_acc: 1.0000
Epoch 62/100
995/995 [==============================] - 0s 47us/step - loss: 0.0061 - acc: 1.0000 - val_loss: 0.0074 - val_acc: 1.0000
Epoch 63/100
995/995 [==============================] -

Epoch 14/100
995/995 [==============================] - 0s 47us/step - loss: 0.0046 - acc: 1.0000 - val_loss: 0.0068 - val_acc: 1.0000
Epoch 15/100
995/995 [==============================] - 0s 49us/step - loss: 0.0045 - acc: 1.0000 - val_loss: 0.0066 - val_acc: 1.0000
Epoch 16/100
995/995 [==============================] - 0s 52us/step - loss: 0.0044 - acc: 1.0000 - val_loss: 0.0064 - val_acc: 1.0000
Epoch 17/100
995/995 [==============================] - 0s 52us/step - loss: 0.0043 - acc: 1.0000 - val_loss: 0.0064 - val_acc: 1.0000
Epoch 18/100
995/995 [==============================] - 0s 50us/step - loss: 0.0043 - acc: 1.0000 - val_loss: 0.0065 - val_acc: 1.0000
Epoch 19/100
995/995 [==============================] - 0s 46us/step - loss: 0.0042 - acc: 1.0000 - val_loss: 0.0065 - val_acc: 1.0000
Epoch 20/100
995/995 [==============================] - 0s 50us/step - loss: 0.0041 - acc: 1.0000 - val_loss: 0.0065 - val_acc: 1.0000
Epoch 21/100
995/995 [==============================] -

Epoch 75/100
995/995 [==============================] - 0s 45us/step - loss: 0.0026 - acc: 1.0000 - val_loss: 0.0057 - val_acc: 1.0000
Epoch 76/100
995/995 [==============================] - 0s 44us/step - loss: 0.0026 - acc: 1.0000 - val_loss: 0.0057 - val_acc: 1.0000
Epoch 77/100
995/995 [==============================] - 0s 45us/step - loss: 0.0026 - acc: 1.0000 - val_loss: 0.0056 - val_acc: 1.0000
Epoch 78/100
995/995 [==============================] - 0s 43us/step - loss: 0.0025 - acc: 1.0000 - val_loss: 0.0056 - val_acc: 1.0000
Epoch 79/100
995/995 [==============================] - 0s 44us/step - loss: 0.0025 - acc: 1.0000 - val_loss: 0.0056 - val_acc: 1.0000
Epoch 80/100
995/995 [==============================] - 0s 45us/step - loss: 0.0025 - acc: 1.0000 - val_loss: 0.0056 - val_acc: 1.0000
Epoch 81/100
995/995 [==============================] - 0s 44us/step - loss: 0.0025 - acc: 1.0000 - val_loss: 0.0056 - val_acc: 1.0000
Epoch 82/100
995/995 [==============================] -

In [11]:


# model.fit(X_train, y_train, validation_data=(X_test,y_test), epochs=epochs_step, batch_size=batch_size, shuffle=shuffle_option)


# # Final evaluation of the model
# scores = model.evaluate(X_test, y_test, verbose=1)
# print("Accuracy: %.2f%%" % (scores[1]*100))

In [14]:

model.fit(X_train, y_train, validation_data=(X_test,y_test), epochs=10, batch_size=20)
scores = model.evaluate(X_test, y_test, verbose=1)
print("Accuracy: %.2f%%" % (scores[1]*100))

Train on 995 samples, validate on 995 samples
Epoch 1/100
995/995 [==============================] - 1s 798us/step - loss: 0.0049 - acc: 0.9990 - val_loss: 0.0081 - val_acc: 0.9990
Epoch 2/100
995/995 [==============================] - 1s 829us/step - loss: 0.0025 - acc: 1.0000 - val_loss: 0.0037 - val_acc: 1.0000
Epoch 3/100
995/995 [==============================] - 1s 761us/step - loss: 0.0016 - acc: 1.0000 - val_loss: 0.0039 - val_acc: 1.0000
Epoch 4/100
995/995 [==============================] - 1s 770us/step - loss: 0.0015 - acc: 1.0000 - val_loss: 0.0040 - val_acc: 1.0000
Epoch 5/100
995/995 [==============================] - 1s 761us/step - loss: 0.0014 - acc: 1.0000 - val_loss: 0.0032 - val_acc: 1.0000
Epoch 6/100
995/995 [==============================] - 1s 750us/step - loss: 0.0013 - acc: 1.0000 - val_loss: 0.0032 - val_acc: 1.0000
Epoch 7/100
995/995 [==============================] - 1s 769us/step - loss: 0.0012 - acc: 1.0000 - val_loss: 0.0030 - val_acc: 1.0000
Epoch 8/1

995/995 [==============================] - 1s 787us/step - loss: 1.8171e-04 - acc: 1.0000 - val_loss: 0.0015 - val_acc: 1.0000
Epoch 60/100
995/995 [==============================] - 1s 764us/step - loss: 1.8515e-04 - acc: 1.0000 - val_loss: 0.0013 - val_acc: 1.0000
Epoch 61/100
995/995 [==============================] - 1s 809us/step - loss: 1.9515e-04 - acc: 1.0000 - val_loss: 0.0012 - val_acc: 1.0000
Epoch 62/100
995/995 [==============================] - 1s 783us/step - loss: 1.8090e-04 - acc: 1.0000 - val_loss: 0.0015 - val_acc: 1.0000
Epoch 63/100
995/995 [==============================] - 1s 799us/step - loss: 1.8615e-04 - acc: 1.0000 - val_loss: 0.0013 - val_acc: 1.0000
Epoch 64/100
995/995 [==============================] - 1s 814us/step - loss: 1.6082e-04 - acc: 1.0000 - val_loss: 0.0019 - val_acc: 1.0000
Epoch 65/100
995/995 [==============================] - 1s 853us/step - loss: 1.7034e-04 - acc: 1.0000 - val_loss: 0.0016 - val_acc: 1.0000
Epoch 66/100
995/995 [===========

In [13]:
#print(X_test[0:4,:,:].shape)
print(np.argmax(model.predict(X_test[1:56,:,:]),axis=1))
print(y_test[1:56])

[3 1 2 2 1 1 1 1 0 0 2 0 3 0 2 2 0 1 3 0 1 2 2 2 0 1 2 0 3 0 1 1 3 1 0 2 2
 1 0 3 1 3 3 3 0 1 3 0 2 2 2 0 3 3 2]
[3 1 2 2 1 1 1 1 0 0 2 0 3 0 2 2 0 1 3 0 1 2 2 2 0 1 2 0 3 0 1 1 3 1 0 2 2
 1 0 3 1 3 3 3 0 1 3 0 2 2 2 0 3 3 2]
